In [1]:
from helpFunctions import get_string_representation_of_kernel as gsr, clean_kernel_expression, print_formatted_hyperparameters
import torch
import gpytorch
import matplotlib.pyplot as plt

In [2]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, kernel_text="RBF", weights=None):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()

        if kernel_text == "RBF":
            self.covar_module = gpytorch.kernels.RBFKernel()
        elif kernel_text == "SIN":
            self.covar_module = gpytorch.kernels.PeriodicKernel()
        elif kernel_text == "SIN+RBF":
            if weights is None:
                self.covar_module = gpytorch.kernels.PeriodicKernel() + gpytorch.kernels.RBFKernel()
            else:
                self.covar_module = weights[0]*gpytorch.kernels.PeriodicKernel() + weights[1]*gpytorch.kernels.RBFKernel()
        elif kernel_text == "SIN*RBF":
            self.covar_module = gpytorch.kernels.PeriodicKernel() * gpytorch.kernels.RBFKernel()
        elif kernel_text == "SIN*LIN":
            self.covar_module = gpytorch.kernels.PeriodicKernel() * gpytorch.kernels.LinearKernel()
        elif kernel_text == "absurd":
            self.covar_module = gpytorch.kernels.RBFKernel() * ((gpytorch.kernels.PeriodicKernel() * gpytorch.kernels.LinearKernel()) + (gpytorch.kernels.PeriodicKernel() * gpytorch.kernels.LinearKernel()))

            
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [3]:
class MyGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(MyGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ZeroMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.PeriodicKernel()) + gpytorch.kernels.ScaleKernel(gpytorch.kernels.PeriodicKernel())
          
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [4]:
def get_kernels_in_kernel_expression(kernel_expression):
    """
    returns list of all base kernels in a kernel expression
    """
    if kernel_expression == None:
        return []
    if hasattr(kernel_expression, "kernels"):
        ret = list()
        for kernel in kernel_expression.kernels:
            ret.extend(get_kernels_in_kernel_expression(kernel))
        return ret
    elif kernel_expression._get_name() == "ScaleKernel":
        return get_kernels_in_kernel_expression(kernel_expression.base_kernel)
    elif kernel_expression._get_name() == "GridKernel":
        return get_kernels_in_kernel_expression(kernel_expression.base_kernel)
    else:
        return [kernel_expression]



In [5]:

# training data for model initialization (e.g. 1 point with x=0, y=0) ; this makes initializing the model easier
prior_x = torch.linspace(0,1,1)
prior_y = prior_x
# initialize likelihood and model
data_likelihood = gpytorch.likelihoods.GaussianLikelihood()
data_model = ExactGPModel(prior_x, prior_y, data_likelihood, kernel_text="SIN")
observations_x = torch.linspace(-10, 10, 200)
# Get into evaluation (predictive posterior) mode
data_model.eval()
data_likelihood.eval()

# Make predictions by feeding model through likelihood
with torch.no_grad(), gpytorch.settings.prior_mode(True):
    observed_pred_prior = data_likelihood(data_model(observations_x))
    f_preds = data_model(observations_x)
    mean_prior = observed_pred_prior.mean
    lower_prior, upper_prior = observed_pred_prior.confidence_region()

f_mean = f_preds.mean
f_var = f_preds.variance
f_covar = f_preds.covariance_matrix
observations_y = f_preds.sample()           # samples from the model


X = observations_x[int((1-0.5)*0.5*200):int((1+0.5)*0.5*200)]
Y = observations_y[int((1-0.5)*0.5*200):int((1+0.5)*0.5*200)]

#X = (X-X.mean())/X.std()
#Y = (Y-Y.mean())/Y.std()


/opt/anaconda3/envs/sage/lib/python3.9/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/opt/anaconda3/envs/sage/lib/python3.9/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(
/opt/anaconda3/envs/sage/lib/python3.9/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-04 to the diagonal
  warnings.warn(
/opt/anaconda3/envs/sage/lib/python3.9/site-packages/linear_operator/operators/_linear_operator.py:2014: NumericalWarning: Runtime Error when computing Cholesky decomposition: Matrix not positive definite after repeatedly adding jitter up to 1.0e-04.. Using symeig method.
  warnings.warn(


In [6]:
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = MyGPModel(X, Y, likelihood)

In [7]:
limits = {"RBFKernel": {"lengthscale": [1e-4,1]},
          "LinearKernel": {"variance": [1e-4,1]},
          "PeriodicKernel": {"lengthscale": [1e-4,10],
                             "period_length": [1e-4,10]},
          "ScaleKernel": {"outputscale": [1e-4,100]},
          "WhiteNoiseKernel": {'lengthscale': [1e-4,1]},
          "CosineKernel": {"period_length": [1e-4,10]},
          "Noise": [1e-4,2e-3],
          "Mean": [0.0,1.0]}


In [8]:
random_repeats = 1000
training_iter = 100

# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

noise_list = list()

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
for j in range(random_repeats):
    #print(list(model.named_parameters()))
    #print("==============")
    try:
        for i in range(training_iter):
            # Zero gradients from previous iteration
            optimizer.zero_grad()
            # Output from model
            output = model(X)
            # Calc loss and backprop gradients
            loss = -mll(output, Y)
            loss.backward()
            optimizer.step()
        noise_list.append(likelihood.noise_covar.noise.item())
        #likelihood.noise_covar.noise = torch.rand(1) * (limits["Noise"][1] - limits["Noise"][0]) + limits["Noise"][0]
        #likelihood.noise_covar.noise = torch.rand(1) + 0.001
        log_loss = -loss * model.train_inputs[0].numel()
        2*log_loss + 2*sum(p.numel() for p in model.parameters() if p.requires_grad)
        for kernel in get_kernels_in_kernel_expression(model.covar_module):
            hypers = limits[kernel._get_name()]
            for hyperparameter in hypers:
                new_value = torch.rand(1) * (hypers[hyperparameter][1] - hypers[hyperparameter][0]) + hypers[hyperparameter][0]
                setattr(kernel, hyperparameter, new_value)
    except:
        print(j)
        break

In [12]:
import numpy as np
print(np.max(noise_list))
print(np.min(noise_list))
print(np.mean(noise_list))
print(np.std(noise_list))

0.6389451622962952
0.023219872266054153
0.29236135746724906
0.0254241432857154
